# Fitting Functions

*Purpose*: **TODO**.


## Setup


In [ ]:
import grama as gr
DF = gr.Intention()
%matplotlib inline

# Fitting

Recall that there are four classes of verb in grama; *fitting* verbs take data as an input and produce a model as an output. There are various ways to fit a model; in this exercise we'll focus on fitting a *parameterized* model using  *nonlinear least squares*.



![Grama verb class diagram](./images/verb-classes.png)


## Trajectory Data


In [ ]:
from grama.data import df_trajectory_windowed
(
    df_trajectory_windowed
    >> gr.ggplot(gr.aes("x", "y"))
    + gr.geom_point()
)

# EMA of Proposed Model


## Trajectory model


In [ ]:
from grama.models import make_trajectory_linear
md_trajectory = make_trajectory_linear()
md_trajectory

### Linear drag model

The drag $\vec{F}_d$ acting on the projectile in this model has a magnitude equal to

$$F_d \propto - m \tau s$$

where $m$ is the mass of the projectile, $s$ is its speed, and $\tau$ is the *time constant* associated with the drag law.

We can make sense of the model by inspecting the equations in the model. However, a complementary way to gain understanding is to do a little *exploratory model analysis*. You'll use EMA below to understand the model's behavior.


## Sinews and bounds

One "gotcha" with `ev_sinews()` is that it can only sweep inputs with *finite bounds*. For variables with one-sided bounds, such as `[0.1, inf]`, we cannot sweep values all the way to infinity! Instead, grama will set the inputs with one-sided bounds to their lower bound, and sweep the remaining variables. We need to set finite bounds in order to use `gr.ev_sinews()`.


### __qX__ Sweep values of `tau`

Update the model to enable a sweep over values of `tau`. How does changing `tau` affect the trajectory? Answer the questions under *observations* below.

*Hint*: Remember that `gr.cp_bounds()` allows you to adjust the bounds of a grama model. Note that you may need to adjust the bounds of time `t` as well as `tau` to make an informative plot.


In [ ]:
(
    md_trajectory

    >> gr.ev_sinews(df_det="swp", n_sweeps=4)
    >> gr.pt_auto()
)

*Observations*

- Once you get a sweep of `tau` displayed, what do each of the curves in the `tau` column represent?
  - (Your response here)
- What affect does `tau` have on the trajectory range `x`?
  - (Your response here)
- What affect does `tau` have on the trajectory height `y`?
  - (Your response here)


### __qX__ Plot a few trajectories

Complete the code below to sweep over values of `tau` and `t`. Compare these results with what you found above. Answer the questions under *observations* below.


In [ ]:
# TASK: Sweep tau and t to create trajectories
(
    md_trajectory
    >> gr.ev_df(
        df=gr.df_grid(
            u0=0.1,
            v0=0.1,
            # TASK: Complete the code to sweep tau and t

        )
    )
    >> gr.ggplot(gr.aes("x", "y", color="factor(tau)"))
    + gr.geom_line()
)

*Observations*

- What affect does `tau` have on the trajectory?
  - (Your response here)
- Imagine that `y == 0` represents the ground. Are the endpoints of these trajectories physically reasonable?
  - (Your response here)


# Fitting with Least Squares

(Above, we guessed at parameter values. Now let's use fitting to get reasonable values.)


## The `ft_nls()` routine


### __qX__ Run `ft_nls()`


In [ ]:
# TASK: 
md_fit = (
    df_trajectory_windowed
    >> gr.ft_nls(md=md_trajectory)
)

md_fit

*Observations*

- ...
  - (Your response here)


### __qX__ Assess the fit


In [ ]:
# TASK: Compare the model predictions with
# the original data df_trajectory_windowed
(
    md_fit

)

*Observations*

- ...
  - (Your response here)


## Rough estimation

Let's do some back-of-the envelope calculations to get a rough sense of reasonable parameter values.

### TODO

$$u = \frac{dx}{dt} \approx \frac{u(t_2) - u(t_1)}{t_2 - t_1}$$

The `gr.lead()` function allows us to access values in the following row; we can use this to implement differences like $u(t_2) - u(t_1)$.

$$F_d/m \propto - \tau s$$

or

$$a_d / s \propto \tau.$$

Note that this is *not* the total acceleration! This is just the acceleration due to drag, which we can't access from the data alone. However, we can use the *total* acceleration to set a reasonable first-guess for the value of `tau`.


In [ ]:
# NOTE: No need to edit
(
    df_trajectory_windowed
    # Estimate velocity components
    >> gr.tf_mutate(
        u=(gr.lead(DF.x) - DF.x) / (gr.lead(DF.t) - DF.t),
        v=(gr.lead(DF.y) - DF.y) / (gr.lead(DF.t) - DF.t),
    )
    # Compute speed
    >> gr.tf_mutate(s=gr.sqrt(DF.u**2 + DF.v**2))
    # Estimate acceleration
    >> gr.tf_mutate(
        a=(gr.lead(DF.s) - DF.s) / (gr.lead(DF.t) - DF.t)
    )
    # Estimate drag time constant
    >> gr.tf_mutate(tau=gr.abs(DF.a / DF.s))
    
    >> gr.tf_select("u", "v", "tau")
    >> gr.tf_describe()
)

### __qX__ Choose reasonable initial parameters

Override the default initial parameter guess of `ft_nls()` by adding a keyword argument. Using the rough estimates above, you should be able to achieve a fit of the data that is quite reasonable. Answer the questions under *observations* below.

*Hint*: Remember to consult the documentation for `ft_nls()` to see how to use its arguments!


In [ ]:
# TASK: 
md_fit_init = (
    df_trajectory_windowed
    >> gr.ft_nls(
        md=md_trajectory,

)

(
    md_fit_init
    >> gr.ev_df(df_trajectory_windowed)
    
    >> gr.ggplot(gr.aes("x", "y"))
    + gr.geom_line()
    + gr.geom_point(data=df_trajectory_windowed)
)

*Observations*

- How well does the trend fit the data?
  - (Your response here)
- How does your initial guess for `tau` compare with the fitted value? What might account for this?
  - (Your response here)
- How does your initial guess for `u0` compare with the fitted value? What might account for this?
  - (Your response here)


# Model Assessment


## Quantifying uncertainty


In [ ]:
# NOTE:  
md_fit_uq = (
    df_trajectory_windowed
    >> gr.ft_nls(
        md=md_trajectory,
        df_init=gr.df_make(
            u0=18,
            v0=18,
            tau=16,
        ),
        uq_method="linpool",
    )
)
md_fit_uq 

### __qX__ Inspect plausible parameter values


In [ ]:
# TASK:
(
    md_fit_uq
    >> gr.ev_sample(n=1e3, df_det="nom", skip=True)
    >> gr.pt_auto()
)

*Observations*

- ...
  - (Your response here)


### __qX__ Inspect plausible trajectories


In [ ]:
# TASK:
(
    md_fit_uq

    
    >> gr.ggplot(gr.aes("x", "y"))
    # NOTE: The `group` aesthetic allows us to draw
    # individual lines for each value of u0; otherwise
    # the lines would all be connected
    + gr.geom_line(gr.aes(group="u0"), alpha=1/5, color="grey")
    # Add the data
    + gr.geom_point(data=df_trajectory_windowed)
)

*Observations*

- According to the fitted model, what is a plausible final range for the projectile? (In what band of `x` values does it hit the ground at `y == 0`?)
  - (Your response here)


# Model Validation

## Validation data


In [ ]:
from grama.data import df_trajectory_full
(
    df_trajectory_full
    >> gr.ggplot(gr.aes("x", "y"))
    + gr.geom_point()
)

### __qX__ Compare the fit to validation data




In [ ]:
# TASK: Compare the fitted model to the validation data;
# make sure to quantify the parameter uncertainty
(
    md_fit_uq
 
)

*Observations*

- ...
  - (Your response here)
